In [4]:
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt

ImportError: No module named tensorflow

In [6]:
# import zipfile

In [7]:
# zip_ref = zipfile.ZipFile('dataset-resized.zip', 'r')
# zip_ref.extractall('datasets')
# zip_ref.close()

In [8]:
base_dir = './dataset-resized'

In [9]:
print(base_dir)

./dataset-resized


In [10]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.


In [11]:
for image_batch, label_batch in train_generator:
    break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 6))

In [12]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
    f.write(labels)

{'cardboard': 0, 'glass': 1, 'metal': 2, 'paper': 3, 'plastic': 4, 'trash': 5}


In [13]:
!cat labels.txt

cardboard
glass
metal
paper
plastic
trash

In [14]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

W0624 22:16:48.816998 140368672560768 deprecation.py:506] From /usr/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


AttributeError: 'Tensor' object has no attribute '_keras_shape'

In [27]:
base_model.trainable = False

In [33]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(6, activation='softmax')
])

In [34]:
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 2,626,854
Trainable params: 368,870
Non-trainable params: 2,257,984
_________________________________________________________________


In [36]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 4


In [37]:
epochs = 10

history = model.fit(train_generator, 
                    epochs=epochs, 
                    validation_data=val_generator)

Epoch 1/10
32/32 [==============================] - 495s 15s/step - loss: 1.4951 - acc: 0.5208 - val_loss: 1.2208 - val_acc: 0.5169
Epoch 2/10
32/32 [==============================] - 490s 15s/step - loss: 0.8088 - acc: 0.7169 - val_loss: 1.8011 - val_acc: 0.4553
Epoch 3/10
32/32 [==============================] - 486s 15s/step - loss: 0.6015 - acc: 0.8039 - val_loss: 1.6426 - val_acc: 0.4751
Epoch 4/10
32/32 [==============================] - 487s 15s/step - loss: 0.4733 - acc: 0.8315 - val_loss: 1.5396 - val_acc: 0.4791
Epoch 5/10
32/32 [==============================] - 500s 16s/step - loss: 0.4223 - acc: 0.8434 - val_loss: 1.6196 - val_acc: 0.5089
Epoch 6/10
32/32 [==============================] - 510s 16s/step - loss: 0.3643 - acc: 0.8661 - val_loss: 1.5327 - val_acc: 0.5447
Epoch 7/10
32/32 [==============================] - 495s 15s/step - loss: 0.3101 - acc: 0.8958 - val_loss: 2.3427 - val_acc: 0.4533
Epoch 8/10
32/32 [==============================] - 492s 15s/step - loss: 0.

In [46]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

KeyError: 'accuracy'

In [47]:
base_model.trainable = True

In [48]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

Number of layers in the base model:  155


In [49]:
model.compile(loss='categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

In [50]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 2,626,854
Trainable params: 2,231,462
Non-trainable params: 395,392
_________________________________________________________________


In [51]:
print('Number of trainable variables = {}'.format(len(model.trainable_variables)))

Number of trainable variables = 60


In [44]:
history_fine = model.fit(train_generator, 
                         epochs=5,
                         validation_data=val_generator)

Epoch 1/5
32/32 [==============================] - 661s 21s/step - loss: 0.1658 - acc: 0.9541 - val_loss: 1.9630 - val_acc: 0.5328
Epoch 2/5
32/32 [==============================] - 641s 20s/step - loss: 0.1167 - acc: 0.9738 - val_loss: 1.9574 - val_acc: 0.5368
Epoch 3/5
32/32 [==============================] - 642s 20s/step - loss: 0.1110 - acc: 0.9768 - val_loss: 1.9886 - val_acc: 0.5328
Epoch 4/5
32/32 [==============================] - 682s 21s/step - loss: 0.0937 - acc: 0.9852 - val_loss: 1.9787 - val_acc: 0.5368
Epoch 5/5
32/32 [==============================] - 682s 21s/step - loss: 0.0803 - acc: 0.9886 - val_loss: 1.9721 - val_acc: 0.5368


In [45]:
saved_model_dir = 'save/fine_tuning'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

ValueError: None values not supported.

In [ ]:
#files.download('model.tflite')
#files.download('labels.txt')

In [ ]:
acc = history_fine.history['accuracy']
val_acc = history_fine.history['val_accuracy']

loss = history_fine.history['loss']
val_loss = history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()